In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

# Define the base URL of the website you want to scrape
base_url = "https://zinnov.com/"

# Initialize a set to store visited URLs
visited_urls = set()

# Function to check if it's allowed to scrape a URL based on robots.txt
def is_allowed_by_robots_txt(url):
    # Get the URL of the robots.txt file
    robots_url = urljoin(url, "/robots.txt")
    
    # Send an HTTP GET request to the robots.txt file
    robots_response = requests.get(robots_url)
    
    # Check if the request was successful (status code 200)
    if robots_response.status_code == 200:
        # Check if the robots.txt file allows user-agent "User-agent: *"
        if "User-agent: *" in robots_response.text:
            # Check if "Disallow:" is specified for the current URL
            for line in robots_response.text.split('\n'):
                if line.strip().startswith("Disallow:"):
                    disallowed_path = line.split(":")[1].strip()
                    if disallowed_path == "/":
                        return False
                    if url.endswith("/"):
                        url = url[:-1]
                    if disallowed_path == url[len(base_url):]:
                        return False
        return True
    else:
        # If there's no robots.txt file or if there's an issue with it, assume it's allowed
        return True

# Function to recursively scrape all pages
def scrape_page(url):
    if url in visited_urls:
        return
    
    visited_urls.add(url)
    
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Save the HTML content to a file in a subdirectory
        relative_path = url[len(base_url):]
        if relative_path == "":
            relative_path = "index.html"
        file_path = os.path.join("scraped_data", relative_path)
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(soup.prettify())
        
        print(f"Scraped: {url}")
        
        # Find all links on the page and recursively scrape them
        links = soup.find_all('a', href=True)
        for link in links:
            next_url = urljoin(url, link['href'])
            if next_url.startswith(base_url) and is_allowed_by_robots_txt(next_url):
                scrape_page(next_url)
    else:
        print(f"Failed to retrieve: {url}")

# Create a directory to store the scraped data
os.makedirs("scraped_data", exist_ok=True)

# Start scraping from the base URL
scrape_page(base_url)

print("Scraping completed.")


Scraped: https://zinnov.com/


PermissionError: [Errno 13] Permission denied: 'scraped_data\\about-us/news-media/'